In [ ]:
import os
import bioimage

from interactive_m2unet import M2UnetInteractiveModel
import numpy as np
import imageio
import albumentations as A
from skimage.filters import threshold_otsu
from skimage.measure import label
# Uncomment to specify the gpu number
# os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import torch
torch.backends.cudnn.benchmark = True

TRAIN = True
TEST = False




In [ ]:
# a function for loading cellpose output (image, mask and outline)
def load_samples(train_dir):
    npy_files = [os.path.join(train_dir, s) for s in os.listdir(train_dir) if s.endswith('.npy')]
    samples = []
    for file in npy_files:
        print(file)
        try:
            items = np.load(file, allow_pickle=True).item()
        except:
            print("Bad Item")
            continue
        mask = (items['masks'][:, :, None]  > 0) * 1.0
        outline = (items['outlines'][:, :, None]  > 0) * 1.0
        mask = mask * (1.0 - outline)
        sample = (items['img'], mask)
        samples.append(sample)
    return samples

# check if GPU is available
print(f'GPU: {torch.cuda.is_available()}')

# setting up
data_dir = './cell_data_3' # data should contain a train and a test folder
model_root = "./models_100"
epochs = 2
steps = 1
resume = True
corrid = "200"
pretrained_model = None  # os.path.join(model_root, str(corrid), "model.h5")
os.makedirs(os.path.join(model_root, str(corrid)), exist_ok=True)
sz = 2048
sz_outer = int(sz* 1.5)

# define the transforms
transform = A.Compose(
    [
        A.RandomCrop(sz, sz),
        #A.Rotate(limit=[-5, 5], p=1),
        A.Flip(p=0.5),
        #A.CenterCrop(sz, sz),
    ]
)
A.save(transform, "./models/transform.json")
# unet model hyperparamer can be found here: https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=f899f7a8a9144b3f946c4a1362f7e38ae0c00c59&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f79696e676b61697368612f6b657261732d756e65742d636f6c6c656374696f6e2f663839396637613861393134346233663934366334613133363266376533386165306330306335392f6578616d706c65732f757365725f67756964655f6d6f64656c732e6970796e62&logged_in=true&nwo=yingkaisha%2Fkeras-unet-collection&path=examples%2Fuser_guide_models.ipynb&platform=mac&repository_id=323426984&repository_type=Repository&version=95#Swin-UNET
model_config = {
    "type": "m2unet",
    "activation": "sigmoid",
    "output_channels": 1,
    "loss": {"name": "BCELoss", "kwargs": {}},
    "optimizer": {"name": "RMSprop", "kwargs": {"lr": 1e-2, "weight_decay": 1e-8, "momentum": 0.9}},
    "augmentation": A.to_dict(transform),
}
model = M2UnetInteractiveModel(
    model_config=model_config,
    model_dir=model_root,
    resume=resume,
    pretrained_model=pretrained_model,
    default_save_path=os.path.join(model_root, str(corrid), "model.pth"),
)

# load samples
train_samples = load_samples(data_dir + '/train')
test_samples = load_samples(data_dir + '/test')

# train the model 
if TRAIN:
    iterations = 0
    for epoch in range(epochs):
        print('epoch: ')
        print(epoch)
        losses = []
        # image shape: sz, sz, 3
        # labels shape: sz, sz, 1
        for (image, labels) in train_samples:
            mask = model.transform_labels(labels)
            x = np.expand_dims(image, axis=0)
            y = np.expand_dims(mask, axis=0)
            losses = []
            for _ in range(steps):
                # x and y will be augmented for each step
                loss = model.train_on_batch(x, y)
                losses.append(loss)
                iterations += 1
                print(f"iteration: {iterations}, loss: {loss}")
    model.save()

# test
if TEST:
    for i, sample in enumerate(test_samples):
        inputs = sample[0].astype("float32")[None, :sz, :sz, :]
        imageio.imwrite(f"octopi-inputs_{i}.png", inputs[0].astype('uint8'))
        labels = sample[1].astype("float32")[None, :sz, :sz, :] * 255
        imageio.imwrite(f"octopi-labels_{i}.png", labels[0].astype('uint8'))
        results = model.predict(inputs)
        output = np.clip(results[0] * 255, 0, 255)[:, :, 0].astype('uint8')
        imageio.imwrite(f"octopi-pred-prob_{i}.png", output)
        threshold = threshold_otsu(output)
        mask = ((output > threshold) * 255).astype('uint8')
        predict_labels = label(mask)
        imageio.imwrite(f"octopi-pred-labels_{i}.png", predict_labels)

    print("all done")
    



In [ ]:
# Test
TEST = True
TRAIN = True

import numpy as np
import imageio
import albumentations as A
from skimage.filters import threshold_otsu
from skimage.measure import label
# Uncomment to specify the gpu number
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import torch
torch.backends.cudnn.benchmark = True

# a function for loading cellpose output (image, mask and outline)
def load_samples(train_dir):
    npy_files = [os.path.join(train_dir, s) for s in os.listdir(train_dir) if s.endswith('.npy')]
    samples = []
    for file in npy_files:
        print(file)
        try:
            items = np.load(file, allow_pickle=True).item()
        except:
            print("Bad Item")
            continue
        mask = (items['masks'][:, :, None]  > 0) * 1.0
        outline = (items['outlines'][:, :, None]  > 0) * 1.0
        mask = mask * (1.0 - outline)
        sample = (items['img'], mask)
        samples.append(sample)
    return samples

# check if GPU is available
print(f'GPU: {torch.cuda.is_available()}')

# setting up
data_dir = './cell_data_3' # data should contain a train and a test folder
model_root = "./models_100"
epochs = 1
steps = 1
resume = True
corrid = "200"
pretrained_model = None  # os.path.join(model_root, str(corrid), "model.h5")
os.makedirs(os.path.join(model_root, str(corrid)), exist_ok=True)
sz = 2048
sz_outer = int(sz* 1.5)

# define the transforms
transform = A.Compose(
    [
        A.RandomCrop(sz, sz),
        #A.Rotate(limit=[-5, 5], p=1),
        A.Flip(p=0.5),
        #A.CenterCrop(sz, sz),
    ]
)
A.save(transform, "./models/transform.json")
# unet model hyperparamer can be found here: https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=f899f7a8a9144b3f946c4a1362f7e38ae0c00c59&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f79696e676b61697368612f6b657261732d756e65742d636f6c6c656374696f6e2f663839396637613861393134346233663934366334613133363266376533386165306330306335392f6578616d706c65732f757365725f67756964655f6d6f64656c732e6970796e62&logged_in=true&nwo=yingkaisha%2Fkeras-unet-collection&path=examples%2Fuser_guide_models.ipynb&platform=mac&repository_id=323426984&repository_type=Repository&version=95#Swin-UNET
model_config = {
    "type": "m2unet",
    "activation": "sigmoid",
    "output_channels": 1,
    "loss": {"name": "BCELoss", "kwargs": {}},
    "optimizer": {"name": "RMSprop", "kwargs": {"lr": 1e-2, "weight_decay": 1e-8, "momentum": 0.9}},
    "augmentation": A.to_dict(transform),
}
model = M2UnetInteractiveModel(
    model_config=model_config,
    model_dir=model_root,
    resume=resume,
    pretrained_model=pretrained_model,
    default_save_path=os.path.join(model_root, str(corrid), "model.pth"),
)

# load samples
train_samples = load_samples(data_dir + '/train')
test_samples = load_samples(data_dir + '/test')

# train the model 
if TRAIN:
    iterations = 0
    for epoch in range(epochs):
        print('epoch: ')
        print(epoch)
        losses = []
        # image shape: sz, sz, 3
        # labels shape: sz, sz, 1
        for (image, labels) in train_samples:
            mask = model.transform_labels(labels)
            x = np.expand_dims(image, axis=0)
            y = np.expand_dims(mask, axis=0)
            losses = []
            for _ in range(steps):
                # x and y will be augmented for each step
                loss = model.train_on_batch(x, y)
                losses.append(loss)
                iterations += 1
                print(f"iteration: {iterations}, loss: {loss}")
    model.save()

# test
if TEST:
    for i, sample in enumerate(test_samples):
        inputs = sample[0].astype("float32")[None, :sz, :sz, :]
        print[inputs[0].shape]
        imageio.imwrite(f"octopi-inputs_{i}.png", inputs[0].astype('uint8'))
        labels = sample[1].astype("float32")[None, :sz, :sz, :] * 255
        imageio.imwrite(f"octopi-labels_{i}.png", labels[0].astype('uint8'))
        results = model.predict(inputs)
        output = np.clip(results[0] * 255, 0, 255)[:, :, 0].astype('uint8')
        imageio.imwrite(f"octopi-pred-prob_{i}.png", output)
        threshold = threshold_otsu(output)
        mask = ((output > threshold) * 255).astype('uint8')
        predict_labels = label(mask)
        imageio.imwrite(f"octopi-pred-labels_{i}.png", predict_labels)

    print("all done")


In [ ]:
np.load('./072622-D5-6_2022-07-27_17-22-25.774065/3_0_f_BF_LED_matrix_dpc_seg.npy', allow_pickle=True).item()

In [ ]:
torch.cuda.is_available()

In [ ]:
# Jaccard Similarity
import numpy as np
import cv2
import glob

labels_str = "octopi-labels_"
pred_str = "octopi-pred-labels_"
imgs = glob.glob("*.png")
n_labels = len([i for i in imgs if labels_str in i])

def jaccard_sim(img1, img2):
    n = np.prod(img1.shape)
    a = img1 * img2
    b = img1 + img2 - a
    J = a/b
    J[np.isnan(J)] = 1
    j = np.sum(J)/n

    return j


j = []
for i in range(n_labels):
    label = labels_str + str(i) + ".png"
    pred  = pred_str + str(i) + ".png"

    i_pred = np.array(cv2.imread(pred)[:,:,0], dtype='f')
    i_label = np.array(cv2.imread(label)[:,:,0], dtype='f')

    i_pred = i_pred/255.0
    i_label = i_label/255.0

    j.append(jaccard_sim(i_label, i_pred))
    
print(j)
print(min(j))
print(max(j))

In [ ]:
## Store Jaccard Similarity

[0.6749610304832458, 0.6765353679656982, 0.6738048791885376, 0.6553453803062439, 0.6521809697151184, 0.6666283011436462, 0.6581857204437256, 0.6607326865196228, 0.6365719437599182, 0.6806446313858032, 0.6748928427696228, 0.6598854064941406, 0.6804280281066895, 0.6643669605255127, 0.6381222605705261, 0.6740015745162964, 0.6641574501991272, 0.6487610936164856, 0.663611650466919, 0.6803819537162781]
0.6365719437599182
0.6806446313858032


[0.669924259185791, 0.6443573832511902, 0.6819045543670654, 0.6344529390335083, 0.6782583594322205, 0.6359854340553284, 0.6348254084587097, 0.6979055404663086, 0.6535060405731201, 0.7098541855812073, 0.6927616000175476, 0.6919836401939392, 0.6912431120872498, 0.6589328050613403, 0.678594708442688, 0.692324161529541, 0.662678062915802, 0.6622503995895386, 0.6850878000259399, 0.7019121050834656]
0.6344529390335083
0.7098541855812073